#NOTAS

https://stackoverflow.com/questions/33541551/hough-lines-in-opencv-python




#Base fetch

In [1]:
%%shell
if [ ! -d base ]
then
  echo "Fetching..."
  git clone https://github.com/ufrpe-mineracao-textos/projeto-de-mineracao-20192-chart-minerators.git
  echo "Copying files..."
  cp -r /content/projeto-de-mineracao-20192-chart-minerators/base base
  cp -r /content/projeto-de-mineracao-20192-chart-minerators/nova_base nova_base
  echo "Done!"
else
  echo "Directory already exists !"
fi


Fetching...
Cloning into 'projeto-de-mineracao-20192-chart-minerators'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 265 (delta 65), reused 129 (delta 3), pack-reused 0
Receiving objects: 100% (265/265), 35.21 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Copying files...
Done!


#Image segmentation with Hough to detect axis
This code aims to segment graphs images between line and axis separately

###Detection axis function
Utilized for experiments

In [0]:
def detect_axis_from_chart_probabilistic(img):
  img_out = img.copy()
  blurGaussian = cv2.GaussianBlur(img,(5,5),0)
  ret, th2 = cv2.threshold(blurGaussian, 50, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  edges = cv2.Canny(th2,100,300,apertureSize = 3)
  laplacian = cv2.Laplacian(th2, cv2.CV_8UC1)

  most_left_vertical_line = (400,0,400,400)
  most_right_vertical_line = (0,0,0,400)
  upper_horizontal_line = (0,400,400,400)
  lower_horizontal_line = (0,0,400,0)

  lines = cv2.HoughLinesP(laplacian, 1, np.pi/180, 200, 100, 100)
  for line in lines:
    for x1,y1, x2,y2 in line:
      line = line[0]

      # vertical line case
      if x1 == x2:
        # find most left vertical line
        x_most_left = most_left_vertical_line[0]
        if x1 < x_most_left:
          most_left_vertical_line = line
        # find most right vertical line
        x_most_right = most_right_vertical_line[0]
        if x1 > x_most_right:
          most_right_vertical_line = line

      # horizontal line case
      elif y1 == y2:
        # find upper horizontal line
        y_most_upper = upper_horizontal_line[1]
        if y1 < y_most_upper:
          upper_horizontal_line = line
        # find lower horizontal line
        y_most_lower = lower_horizontal_line[1]
        if y1 > y_most_lower:
          lower_horizontal_line = line

  x1, y1, x2, y2 = most_left_vertical_line
  cv2.line(img_out, (x1,y1), (x2,y2),(0,0,255),5)
  x1, y1, x2, y2 = most_right_vertical_line
  cv2.line(img_out, (x1,y1), (x2,y2),(0,0,255),5)
  x1, y1, x2, y2 = upper_horizontal_line
  cv2.line(img_out, (x1,y1), (x2,y2),(0,0,255),5)
  x1, y1, x2, y2 = lower_horizontal_line
  cv2.line(img_out, (x1,y1), (x2,y2),(0,0,255),5)

  return img_out

###Mask to isolate lines function

In [0]:
def get_mask_over_lines(img):
  img_out = np.ones((400,400), np.uint8) * 255
  
  blurGaussian = cv2.GaussianBlur(img,(5,5),0)
  ret, th2 = cv2.threshold(blurGaussian, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

  # edges = cv2.Canny(th2,100,300,apertureSize = 3) # TODO removed
  laplacian = cv2.Laplacian(th2, cv2.CV_8UC1)
  kernel = np.array( [[0,1,0],
                      [1,1,1],
                      [0,1,0]], np.uint8)
  laplacian = cv2.dilate(laplacian,kernel,iterations = 1)

  # TODO debug
  # print('mask - th2')
  # cv2_imshow(th2)
  # print('mask - laplacian')
  # cv2_imshow(laplacian) 

  most_left_vertical_line = (400,0,400,400)
  most_right_vertical_line = (0,0,0,400)
  upper_horizontal_line = (0,400,400,400)
  lower_horizontal_line = (0,0,400,0)

  lines = cv2.HoughLinesP(laplacian, 1, np.pi/180, 200, 100, 100)
  for line in lines:
    for x1,y1, x2,y2 in line:
      line = line[0]

      # vertical line case
      if x1 == x2:
        # find most left vertical line
        x_most_left = most_left_vertical_line[0]
        if x1 < x_most_left:
          most_left_vertical_line = line
        # find most right vertical line
        x_most_right = most_right_vertical_line[0]
        if x1 > x_most_right:
          most_right_vertical_line = line

      # horizontal line case
      elif y1 == y2:
        # find upper horizontal line
        y_most_upper = upper_horizontal_line[1]
        if y1 < y_most_upper:
          upper_horizontal_line = line
        # find lower horizontal line
        y_most_lower = lower_horizontal_line[1]
        if y1 > y_most_lower:
          lower_horizontal_line = line

      # TODO debug
      # cv2.line(img, (x1, y1), (x2, y2), (0,0,0), 2)

  # TODO debug
  # print('img')
  # cv2_imshow(img)

  margin_tolerance = 5

  # left
  x1, y1, x2, y2 = most_left_vertical_line
  cv2.rectangle(img_out, (0,0), (x2+margin_tolerance,400), (0,0,0), -1)

  #right
  x1, y1, x2, y2 = most_right_vertical_line
  cv2.rectangle(img_out, (400,400), (x2-margin_tolerance,0), (0,0,0), -1)
  
  #top
  x1, y1, x2, y2 = upper_horizontal_line
  cv2.rectangle(img_out, (0,0), (400,y2+margin_tolerance), (0,0,0), -1)

  #bottom
  x1, y1, x2, y2 = lower_horizontal_line
  cv2.rectangle(img_out, (400,400), (0,y2-margin_tolerance), (0,0,0), -1)

  return img_out

###Function to segment image between lines and axis

In [0]:
def segment_image_lines_and_axis(img):
  # get mask from axis  
  mask = get_mask_over_lines(img)
  # get only line part of graph
  line_part = cv2.bitwise_and(mask, img)
  # get only numbers and characters part of graph
  mask_inv = 255 - mask
  axis_information_part = cv2.bitwise_and(mask_inv, img)

  line_part = cv2.add(line_part, mask_inv)
  axis_information_part = cv2.add(axis_information_part, mask)

  # TODO debug
  # print('before')
  # cv2_imshow(line_part)
  # cv2_imshow(axis_information_part)
  
  line_part = cv2.GaussianBlur(line_part, (5,5), 0)
  ret, th_line_part = cv2.threshold(line_part, 210, 255, cv2.THRESH_BINARY)
  ret, th_axis_part = cv2.threshold(axis_information_part, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

  return th_line_part, axis_information_part

###Execution over files

In [0]:
import math
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import shutil

dest_dir = "base"
# for diretorioPai in glob.glob('base/*'):
for diretorioPai in glob.glob('nova_base/*'):
  for imagem in glob.glob(diretorioPai+'/*'):
    if(".txt" not in imagem and ".docx" not in imagem and ".gif" not in imagem and ".svg" not in imagem and ".png" not in imagem and ".PNG" not in imagem):  
      print(imagem)
      img = cv2.imread(imagem,0)
      img = cv2.resize(img, (400,400))

      print('original')
      cv2_imshow(img)

      line_part, axis_information_part = segment_image_lines_and_axis(img)

      print('processed line part')
      cv2_imshow(line_part)
      print('processed number part')
      cv2_imshow(axis_information_part)

    elif(".gif" in imagem):
      print(imagem)
      im = Image.open(imagem)
      rgb_im = im.convert('RGB')
      path=imagem.split('/')
      nome=path[-1].split('.')
      path_novo = '{}.jpg'.format(nome[0])
      rgb_im.save(path_novo)
      
      img = cv2.imread(path_novo,0)
      img = cv2.resize(img, (400,400))

      print('original')
      cv2_imshow(img)

      line_part, axis_information_part = segment_image_lines_and_axis(img)

      print('processed line part')
      cv2_imshow(line_part)
      print('processed number part')
      cv2_imshow(axis_information_part)

    print(80 * '=')


##Draft
Utilized for experiments

In [0]:
import math
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import shutil

imagem = 'projeto-de-mineracao-20192-chart-minerators/nova_base/Jonas/imagem27.jpg'

print(imagem)
img = cv2.imread(imagem,0)
img = cv2.resize(img, (400,400))

print('original')
cv2_imshow(img)

line_part, axis_information_part = segment_image_lines_and_axis(img)

print('processed line part')
cv2_imshow(line_part)
print('processed number part')
cv2_imshow(axis_information_part)

